In [7]:
!pip install tapi_yandex_metrika

In [8]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
import psycopg2

In [9]:
access_token = 'y0_AgAAAABChhcoAAucBQAAAAEBsv78AACNF9boBKxNFYOQ5VduECOYAH4mAQ'
metric_ids = '65166493'

In [10]:
api = YandexMetrikaStats(
    access_token=access_token
)

In [11]:
from tapi_yandex_metrika import YandexMetrikaStats
import pandas as pd

class YandexMetrikaData:
    def __init__(self, access_token, metric_ids):
        self.access_token = access_token
        self.metric_ids = metric_ids
        self.api = YandexMetrikaStats(access_token=self.access_token)

    def get_data(self, date1, date2, dimensions, metrics, group_by_date=True):
        data_frames = []
        current_date = pd.to_datetime(date1)
        date2 = pd.to_datetime(date2)

        while current_date <= date2:
            params = {
                'ids': self.metric_ids,
                'metrics': metrics,
                'dimensions': dimensions,
                'date1': current_date.strftime('%Y-%m-%d'),
                'date2': date2.strftime('%Y-%m-%d'),
                'sort': 'ym:s:date',
                'filters': "ym:s:isRobot=='No'",
                'accuracy': 'full',
                'limit': 100000
            }

            result = self.api.stats().get(params=params)
            result = result.data

            data_list = []
            for j in range(len(result['data'])):
                dimensions_values = [dim['name'] for dim in result['data'][j]['dimensions']]
                metrics_values = result['data'][j]['metrics']

                row = {**dict(zip(dimensions.split(','), dimensions_values)), **dict(zip(metrics.split(','), metrics_values))}
                data_list.append(row)

            df_temp = pd.DataFrame(data_list)
            data_frames.append(df_temp)

            if len(result['data']) < 100000:
                break

            current_date = pd.to_datetime(result['data'][-1]['dimensions'][0]['name']) + pd.Timedelta(days=1)

            print(f"Date processed: {current_date.strftime('%Y-%m-%d')}")

        df_grouped = pd.concat(data_frames)

        return df_grouped

metrika_data = YandexMetrikaData(access_token, metric_ids)

dimensions = 'ym:s:date,ym:s:ipAddress,ym:s:visitDuration,ym:s:isNewUser,ym:s:daysSinceFirstVisit,ym:s:userVisitsPeriod,ym:s:previousVisitDate,ym:s:ageInterval,ym:s:regionArea,ym:s:userVisits'
metrics = 'ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds,ym:s:pageviews'

date1 = '2019-03-01'
date2 = '2024-03-01'

df_merged = metrika_data.get_data(date1, date2, dimensions, metrics, group_by_date=True)
df_merged


Date processed: 2020-08-20
Date processed: 2020-10-31
Date processed: 2021-02-11
Date processed: 2021-06-24
Date processed: 2021-08-16
Date processed: 2021-11-09
Date processed: 2022-03-03
Date processed: 2022-06-27
Date processed: 2022-08-27
Date processed: 2022-12-28
Date processed: 2023-05-02
Date processed: 2023-07-22
Date processed: 2023-10-15


,ym:s:date,ym:s:ipAddress,ym:s:visitDuration,ym:s:isNewUser,ym:s:daysSinceFirstVisit,ym:s:userVisitsPeriod,ym:s:previousVisitDate,ym:s:ageInterval,ym:s:regionArea,ym:s:userVisits,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds,ym:s:pageviews
0,2020-06-25,109.172.58.xxx,10,New users,0,None,0000-00-00,Age 45-54,Republic of Dagestan,1,0.0,2.0,10.0,2.0
1,2020-06-25,109.196.217.xxx,20,New users,0,None,0000-00-00,Age 35‑44,Krasnoyarsk Krai,1,0.0,2.0,20.0,2.0
2,2020-06-25,109.205.148.xxx,22,New users,0,None,0000-00-00,Not specified,Republic of North Ossetia — Alania,1,0.0,2.0,22.0,2.0
3,2020-06-25,109.205.149.xxx,30,New users,0,None,0000-00-00,Age 18‑24,Republic of North Ossetia — Alania,1,0.0,2.0,30.0,2.0
4,2020-06-25,109.205.149.xxx,283,Returning users,0,0,2020-06-25,Age 18‑24,Republic of North Ossetia — Alania,2,0.0,9.0,283.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97390,2024-03-01,95.161.223.xxx,15,New users,0,None,0000-00-00,Not specified,Saint-Petersburg and Leningrad Oblast,1,0.0,1.0,15.0,1.0
97391,2024-03-01,95.167.247.xxx,14,Returning users,16,16,2024-02-14,Not specified,Chechen Republic,2,0.0,1.0,14.0,1.0
97392,2024-03-01,95.214.211.xxx,259,New users,0,None,0000-00-00,Not specified,Without an area,1,0.0,3.0,259.0,3.0
97393,2024-03-01,95.25.38.xxx,15,Returning users,0,0,2024-02-29,Age 55+,Stavropol Krai,3,0.0,1.0,15.0,1.0
